# Figure 5a
## RTO and RTA for Saccades

In [10]:
import os

import cv2
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from docutils.nodes import legend
from numpy.ma.extras import vstack
from plotly.subplots import make_subplots
import plotly.io as pio

import peyes

import analysis.utils as u
from analysis._article_results._helpers import *
import analysis.statistics.channel_time_diffs as ctd

# pio.renderers.default = "browser"

## Load Data

In [11]:
time_diffs = ctd.load(
    DATASET_NAME,
    PROCESSED_DATA_DIR,
    label=2,        # EventLabelEnum.SACCADE.value
    stimulus_type=STIMULUS_TYPE,
)

## Compute RTO and RTD
**RTO** is the mean difference in onset/offset timings.  
**RTD** is the standard deviation of difference in onset/offset timings.    

Both are measured in samples. 

In [12]:
aggregated_time_diffs = time_diffs.unstack(0).unstack(0).apply(np.hstack, axis=1).apply(lambda vals: vals[~np.isnan(vals)]).rename("time_diffs")
aggregated_time_diffs = aggregated_time_diffs.reindex(u.sort_labelers(aggregated_time_diffs.index.get_level_values(u.PRED_STR)), level=u.PRED_STR)

rto_rtd = pd.concat([
    aggregated_time_diffs.apply(len).rename('N'),
    aggregated_time_diffs.apply(np.mean).rename('RTO'),
    aggregated_time_diffs.apply(np.std).rename('RTD')
], axis=1).unstack(u.PRED_STR)
rto_rtd.index.names = [u.GT_STR, peyes.constants.CHANNEL_TYPE_STR]
rto_rtd.columns.names = [peyes.constants.METRIC_STR, u.PRED_STR]

#### Onsets

In [13]:
rto_rtd.xs(
    peyes.constants.ONSET_STR, level=peyes.constants.CHANNEL_TYPE_STR, axis=0
).drop(columns='N', level=peyes.constants.METRIC_STR).stack(peyes.constants.METRIC_STR, future_stack=True)

pred             RA        MN       ivt      ivvt        idt       idvt  \
gt metric                                                                 
MN RTO     0.029810       NaN  1.270950  1.184783   6.623256   6.399083   
   RTD     1.221612       NaN  1.301823  1.320238  31.873915  33.120040   
RA RTO          NaN -0.029810  1.292070  1.201887   5.431373   6.229032   
   RTD          NaN  1.221612  1.174271  1.223510  27.188705  26.816024   

pred        engbert         nh   remodnav  
gt metric                                  
MN RTO    -0.732620  -1.155709  -2.013333  
   RTD     1.619782  23.123857   2.006611  
RA RTO    -0.750000  -2.436490  -2.679654  
   RTD     1.490083  18.892213  15.103151

#### Offsets

In [14]:
rto_rtd.xs(
    peyes.constants.OFFSET_STR, level=peyes.constants.CHANNEL_TYPE_STR, axis=0
).drop(columns='N', level=peyes.constants.METRIC_STR).stack(peyes.constants.METRIC_STR, future_stack=True)

pred             RA        MN       ivt      ivvt        idt       idvt  \
gt metric                                                                 
MN RTO     0.756098       NaN  3.602241  2.286104  -2.120930  -1.724771   
   RTD     3.035494       NaN  5.995684  5.531914  35.007299  35.637155   
RA RTO          NaN -0.756098  2.711240  1.809074  -2.924590  -2.603226   
   RTD          NaN  3.035494  5.139521  4.508701  29.126477  29.539159   

pred        engbert         nh   remodnav  
gt metric                                  
MN RTO     6.197861   3.049123   8.440000  
   RTD     5.836763  24.778691   6.053627  
RA RTO     5.226277   3.788372   6.424242  
   RTD     5.516720  22.740484  14.681676

## Statistical Analysis

In [15]:
statistics, pvalues, dunns, Ns = ctd.kruskal_wallis_dunns(time_diffs, [GT1, GT2], multi_comp=MULTI_COMP)
statistics.index.names = pvalues.index.names = dunns.index.names = [peyes.constants.CHANNEL_TYPE_STR]

pvalues <= ALPHA

gt,MN,RA
channel_type,,
offset,True,True
onset,True,True


In [16]:
pd.concat([statistics, pvalues], axis=1, keys=['H', 'p']).stack(1, future_stack=True)

H              p
channel_type gt                            
offset       MN   712.615519  1.154953e-150
             RA  1176.835963  4.933279e-251
onset        MN  1298.164616  2.704226e-277
             RA  2078.540777   0.000000e+00

#### Post Hoc Analysis

In [17]:
post_hoc_onset = ctd.post_hoc_table(dunns, peyes.constants.ONSET_STR, [GT1, GT2], alpha=ALPHA)
post_hoc_onset

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.     ***     ***     ***     ***      ***
         RA      --    n.s.     ***     ***     ***     ***      ***
ivvt     MN  1.0000      --     ***     ***     ***     ***      ***
         RA  1.0000      --     ***     ***     ***     ***      ***
idt      MN  0.0000  0.0000      --    n.s.     ***     ***      ***
         RA  0.0000  0.0000      --    n.s.     ***     ***      ***
idvt     MN  0.0000  0.0000  1.0000      --     ***     ***      ***
         RA  0.0000  0.0000  1.0000      --     ***     ***      ***
engbert  MN  0.0000  0.0000  0.0000  0.0000      --     ***        *
         RA  0.0000  0.0000  0.0000  0.0000      --     ***       **
nh       MN  0.0000  0.0000  0.0000  0.0000  0.0000      --      ***
         RA  0.0000  0.0000  0.0000  0.0000  0.0000      --      ***
remodnav MN  0.0000  0.0000  0.0000  0.0000  0.0343  0.0009       --
         RA  0.0000  0.0000  0.0000  0.0000  0.0098  0.0000       --

In [18]:
post_hoc_offset = ctd.post_hoc_table(dunns, peyes.constants.OFFSET_STR, [GT1, GT2], alpha=ALPHA)
post_hoc_offset

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.     ***     ***     ***    n.s.      ***
         RA      --    n.s.     ***     ***     ***    n.s.      ***
ivvt     MN  0.4647      --     ***     ***     ***       *      ***
         RA  0.7420      --     ***     ***     ***     ***      ***
idt      MN  0.0000  0.0000      --    n.s.     ***     ***      ***
         RA  0.0000  0.0000      --    n.s.     ***     ***      ***
idvt     MN  0.0000  0.0000  1.0000      --     ***     ***      ***
         RA  0.0000  0.0000  1.0000      --     ***     ***      ***
engbert  MN  0.0000  0.0000  0.0000  0.0000      --      **     n.s.
         RA  0.0000  0.0000  0.0000  0.0000      --      **        *
nh       MN  1.0000  0.0193  0.0000  0.0000  0.0043      --      ***
         RA  0.4790  0.0004  0.0000  0.0000  0.0019      --      ***
remodnav MN  0.0000  0.0000  0.0000  0.0000  0.0508  0.0000       --
         RA  0.0000  0.0000  0.0000  0.0000  0.0251  0.0000       --

### RTO Distribution
#### (RTD is the s.t.d. around RTO) 

In [19]:
fig = ctd.distributions_figure(time_diffs, GT1, gt2=GT2, only_box=False, show_other_gt=True)
fig.update_layout(
    title=None,
    width=1400, height=500,
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    yaxis=dict(range=[-20, 20]), yaxis2=dict(range=[-20, 20]),
)
fig.show()